In [1]:
from loadmodules import *
import numpy as np
import scipy as sp

from numba import njit, config
config.THREADING_LAYER = 'omp'

In [2]:
def expand_indices(counts):
    return (np.arange(counts.sum()) - np.repeat(np.cumsum(counts) - counts, counts)).astype(np.int64)

def mask_equal_to_previous(arr):
    mask = np.ones(len(arr), dtype=bool)
    mask[1:] = arr[1:] != arr[:-1]
    return mask

def B(x):
    return sp.special.erf(x) * 2.*x*np.exp(-x**2)/np.sqrt(np.pi)

@njit(parallel=True)
def velocity_dispersion(radius, parts_radius, starparts, s_data_age, s_data_vel, s_data_type):
    # Select particles within the given radius
    within_radius = parts_radius[starparts][s_data_age > 0.] < radius
    if within_radius.sum() >= 48:
        velocities = np.sqrt(np.sum(s_data_vel[starparts][s_data_age > 0.][within_radius]**2, axis=1))
    # if not enough stars, use dark matter particles
    else:
        mask_dm = (s_data_type != 4) * (s_data_type != 0)
        within_radius = parts_radius[mask_dm] < radius
        velocities = np.sqrt(np.sum(s_data_vel[mask_dm][within_radius]**2, axis=1))

    # Calculate the velocity dispersion
    if velocities.size == 0:
        return 0.0
    # Avoid division by zero
    else:
        return np.std(velocities)

In [5]:
path = './Au6_lvl4_d8ad_testchanges/output/'

num_snaps = 128
files_per_snap = 8

In [6]:
for i in range(127, num_snaps):
    print('Loading snapshot', i)
    sf = load_subfind(i, dir=path, hdf5=True, loadonly=['fpos', 'frc2', 'svel', 'flty', 'fnsh', 'slty', 'spos', 'smty', 'ffsh'] )
    s = gadget_readsnap(i, snappath=path, subfind=sf, hdf5=True, loadonlyhalo=0)
    print('Redshift:', s.redshift, ' cosmo time:', s.time)

    if ((s.data['type']==4).sum() > 0):
        if((s.data['incl']>0).sum() > 0):
            print('Found stars with GC in this snapshot')
            s.calc_sf_indizes( sf )
            galrad = 0.1 * sf.data['frc2'][0]
            s.select_halo( sf, use_principal_axis=True, use_cold_gas_spin=False, do_rotation=True, verbose=False )

            Gcosmo = 43.
            starparts = s.data['type']==4
            
            kinetic_energy = np.sum(s.data['vel']**2, axis=1)

            orbital_energy = s.data['pot'] + 0.5 * kinetic_energy
            orbital_energy /= 1e5

            e_max = orbital_energy.max()
            orbital_energy -= e_max

            Jtot = np.sqrt((np.cross( s.data['pos'], (s.data['vel'] ))**2).sum(axis=1))
            Lz = np.cross( s.data['pos'], (s.data['vel'] ) )[:,0]
            Lz *= np.sign(np.nanmedian(Lz))

            isort_parts = np.argsort(s.r())
            revert_sort = np.argsort(isort_parts)
            cummass = np.cumsum(s.data['mass'][isort_parts])
            Vc_parts = np.sqrt(Gcosmo*cummass[revert_sort]/s.r())

            # Energy of circular orbits at increasing radii
            Ecirc = 0.5*Vc_parts[isort_parts]**2 + s.data['pot'][isort_parts]
            Ecirc /= 1e5
            Ecirc -= e_max

            mask = mask_equal_to_previous(s.r()[isort_parts][~np.isinf(Ecirc)])

            r_test = np.logspace(-4.5, np.log10(s.r().max()), 500)
            Ecirc_f = sp.interpolate.PchipInterpolator(s.r()[isort_parts][~np.isinf(Ecirc)][mask], Ecirc[~np.isinf(Ecirc)][mask])
            Vc_f = sp.interpolate.PchipInterpolator(s.r()[isort_parts][~np.isinf(Ecirc)][mask], Vc_parts[isort_parts][~np.isinf(Ecirc)][mask])
            Mr_f = sp.interpolate.PchipInterpolator(s.r()[isort_parts][~np.isinf(Ecirc)][mask], cummass[~np.isinf(Ecirc)][mask])
    
            mask_clusters_initial = (s.data['incl'] > 0)
    
            idx = np.argmin(np.abs(orbital_energy[starparts][mask_clusters_initial,np.newaxis] - Ecirc_f(r_test)), axis=1)
            rc = r_test[idx]
            vc = Vc_f(rc)
            Lzmax = rc*vc
            circ_param = Lz[starparts][mask_clusters_initial]/Lzmax
    
            cluster_masses = s.data['mclt'][mask_clusters_initial].flatten()
            init_cluster_masses = s.data['imcl'][mask_clusters_initial].flatten()
            cluster_mlost_sh = s.data['mlsk'][mask_clusters_initial].flatten()
            cluster_mlost_rx = s.data['mlrx'][mask_clusters_initial].flatten()
            not_empty_clusters = (init_cluster_masses > 0.)
            cluster_masses = cluster_masses[not_empty_clusters]
            cluster_mlost_sh = cluster_mlost_sh[not_empty_clusters]
            cluster_mlost_rx = cluster_mlost_rx[not_empty_clusters]
            init_cluster_masses = init_cluster_masses[not_empty_clusters]

            part_id = np.repeat(s.data['id'][starparts], s.data['incl'])
            scs_id = expand_indices(s.data['incl'][mask_clusters_initial])
            
            clusters_formtime = np.repeat(s.data['age'], s.data['incl'])
            clusters_age = s.cosmology_get_lookback_time_from_a(clusters_formtime, is_flat=True) - s.cosmology_get_lookback_time_from_a(s.time, is_flat=True)
            # do the DF timescale estimate for clusters with mass
            mask_mass = (cluster_masses > 0.)
    
            rc_clus = np.repeat(rc, s.data['incl'][mask_clusters_initial])
            M_rc_clus = np.repeat(Mr_f(rc), s.data['incl'][mask_clusters_initial])
            vc_rc_clus = np.repeat(vc, s.data['incl'][mask_clusters_initial])
            sigma_rc_clus = np.zeros_like(rc_clus)
            sigma_clus = np.array([velocity_dispersion(r, s.r(), starparts, s.data['age'], s.data['vel'], s.data['type']) for r in 
                                    rc[s.data['nclt'][mask_clusters_initial]>0]])
            sigma_rc_clus[mask_mass] = np.repeat(sigma_clus, s.data['nclt'][mask_clusters_initial][s.data['nclt'][mask_clusters_initial]>0])

            feps = np.repeat((Jtot[starparts][mask_clusters_initial]/Lzmax)**0.78, s.data['incl'][mask_clusters_initial])
            coulumblog = np.zeros_like(rc_clus)
            coulumblog[mask_mass] = np.log(1. + M_rc_clus[mask_mass]/cluster_masses[mask_mass])
            
            tdf = 2e4 * np.ones_like(rc_clus)
            tdf[mask_mass] = feps[mask_mass]/(2*B(vc_rc_clus[mask_mass]/(np.sqrt(2.)*sigma_rc_clus[mask_mass])))*np.sqrt(2.)*sigma_rc_clus[mask_mass]* \
                            rc_clus[mask_mass]**2./(Gcosmo*cluster_masses[mask_mass]*coulumblog[mask_mass])
            tdf *= s.UnitLength_in_cm/s.UnitVelocity_in_cm_per_s / (1e9*365.25*24*3600)

            mask_disrupted = (tdf < clusters_age)
            if (mask_disrupted.sum() > 0):
                print('Clusters disrupted by dynamical friction {:d}'.format(mask_disrupted.sum()))
                for j in range(i, num_snaps):
                    found = 0
                    k = 0
                    while found < mask_disrupted.sum():
                        h5_file = h5py.File(path + 'snapdir_{:03d}/snapshot_{:03d}.{:d}.hdf5'.format(j,j,k), 'r+')
                        stars = h5_file['PartType4']
                        ids = stars['ParticleIDs'][:]
                        clus_mass = stars['ClusterMass'][:]
                        clus_radius = stars['ClusterRadius'][:]
                        disruption_time = stars['DisruptionTime'][:]
                        mlost_shocks = stars['MassLostShocks'][:]
                        mlost_relax = stars['MassLostRelaxation'][:]
                        nclus = stars['NumberOfClusters'][:]
                        inverse_mask = np.isin(part_id[mask_disrupted], ids)
                        found += inverse_mask.sum()
                        if inverse_mask.sum()>0:
                            print('Found {:d} clusters in snapshot {:d} part {:d}'.format(inverse_mask.sum(), j, k))
                            for cl_idx in range(inverse_mask.sum()):
                                mask_id = np.isin(ids, part_id[mask_disrupted][inverse_mask][cl_idx])
                                clus_mass[mask_id, scs_id[mask_disrupted][inverse_mask][cl_idx]] = 0.0
                                clus_radius[mask_id, scs_id[mask_disrupted][inverse_mask][cl_idx]] = 0.0
                                mlost_shocks[mask_id, scs_id[mask_disrupted][inverse_mask][cl_idx]] = cluster_mlost_sh[mask_disrupted][inverse_mask][cl_idx]
                                mlost_relax[mask_id, scs_id[mask_disrupted][inverse_mask][cl_idx]] = cluster_mlost_rx[mask_disrupted][inverse_mask][cl_idx]
                                nclus[mask_id] = (clus_mass[mask_id] > 0.).sum()
                                disruption_time[mask_id, scs_id[mask_disrupted][inverse_mask][cl_idx]] = s.time
                        stars['ClusterMass'][:] = clus_mass
                        stars['ClusterRadius'][:] = clus_radius
                        stars['DisruptionTime'][:] = disruption_time
                        stars['MassLostShocks'][:] = mlost_shocks
                        stars['MassLostRelaxation'][:] = mlost_relax
                        stars['NumberOfClusters'][:] = nclus
                        h5_file.close()
                        k+=1
            else:
                print('No clusters disrupted by dynamical friction in this snapshot')
        else:
            print('NO STARS WITH GC IN MAIN HALO IN THIS SNAPSHOT')
    else:
        print('No stars in this snapshot')

Loading snapshot 127


/users/aripcont/pylib/gadget_snap.py:732: H5pyDeprecationWarning: Using astype() as a context manager is deprecated. Slice the returned object instead, like: ds.astype(np.int32)[:10]
  with dset.astype('uint64'):


Redshift: 2.220446049250313e-16  cosmo time: 0.9999999999999998
Found stars with GC in this snapshot


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


Clusters disrupted by dynamical friction 1
Found 1 clusters in snapshot 127 part 0
